Изучите таблицу `airports` и выведите список городов (`city`), в которых есть аэропорты.  

In [ ]:
'''
SELECT
    DISTINCT city
FROM
    airports
'''

Изучите таблицу `flights` и подсчитайте количество вылетов (`flight_id`) из каждого аэропорта вылета (`departure_airport`). Назовите переменную `cnt_flights` и выведите её вместе со столбцом `departure_airport` — сначала `departure_airport`, потом `cnt_flights`. Результат отсортируйте в порядке убывания количества вылетов.

In [ ]:
'''
SELECT
    departure_airport,
    COUNT(flight_id) as cnt_flights
FROM
    flights
GROUP BY
    departure_airport
ORDER BY
    COUNT(flight_id) DESC
'''

Найдите количество рейсов на каждой модели самолёта с вылетом в сентябре 2018 года. Назовите получившийся столбец `flights_amount` и выведите его, также выведите столбец `model`.

In [ ]:
'''
SELECT 
    aircrafts.model as model,
    COUNT(flights.flight_id) AS flights_amount
FROM
    flights
    LEFT JOIN aircrafts ON aircrafts.aircraft_code = flights.aircraft_code
WHERE
    CAST(flights.departure_time as date) > '2018-08-31' 
    AND CAST(flights.departure_time as date) < '2018-10-01'
GROUP BY
    aircrafts.model
'''

Посчитайте количество рейсов по всем моделям самолётов Boeing, Airbus и другим (`'other'`) в сентябре. Типы моделей поместите в столбец `type_aircraft`, а количество рейсов — во `flights_amount`. Выведите их на экран.

In [ ]:
'''
SELECT
    CASE WHEN aircrafts.model LIKE '%Boeing%' THEN
        'Boeing'
    WHEN aircrafts.model LIKE '%Airbus%' THEN    
        'Airbus'
    ELSE
        'other'
    END AS type_aircraft,
    COUNT(flights.flight_id) as flights_amount
FROM
    flights
    INNER JOIN aircrafts ON aircrafts.aircraft_code = flights.aircraft_code
WHERE
    CAST(flights.departure_time as date) > '2018-08-31' 
    AND CAST(flights.departure_time as date) < '2018-10-01'
GROUP BY
    type_aircraft
'''

Посчитайте среднее количество прибывающих рейсов в день для каждого города за август 2018 года. Назовите получившееся поле `average_flights`, вместе с ним выведите столбец `city`.
Выводите столбцы в таком порядке:
* city,
* average_flights.

In [ ]:
'''
SELECT
    subq.city as city,
    AVG(qty) AS average_flights
FROM (
        SELECT
            airports.city as city,
            CAST(flights.arrival_time AS date) AS ar_date,
            count(flights.arrival_time) AS qty
        FROM
            flights
            INNER JOIN airports ON airports.airport_code = flights.arrival_airport
        WHERE
            EXTRACT(MONTH FROM flights.arrival_time) = 8
        GROUP BY
            city,
            ar_date) AS subq
GROUP BY
    city
'''

Установите фестивали, которые проходили с 23 июля по 30 сентября 2018 года в Москве, и номер недели, в которую они проходили. Выведите название фестиваля `festival_name` и номер недели `festival_week`.

In [ ]:
'''
SELECT
    festival_name,
    EXTRACT(WEEK FROM festival_date) AS festival_week
FROM
    festivals
WHERE
    festival_city = 'Москва'
    AND festival_date BETWEEN '2018-07-23' AND '2018-09-30'
'''

Для каждой недели с 23 июля по 30 сентября 2018 года посчитайте количество билетов, купленных на рейсы в Москву (номер недели `week_number` и количество билетов `ticket_amount`). Получите таблицу, в которой будет информация о количестве купленных за неделю билетов; отметка, проходил ли в эту неделю фестиваль; название фестиваля `festival_name` и номер недели `week_number`.
Выводите столбцы в таком порядке: - `week_number` - `ticket_amount` - `festival_week` - `festival_name`

In [ ]:
'''
SELECT
    T.week_number,
    T.ticket_amount,
    T.festival_week,
    T.festival_name
FROM ((
        SELECT
            EXTRACT(week FROM flights.departure_time) AS week_number,
            COUNT(ticket_flights.ticket_no) AS ticket_amount
        FROM
            airports
            INNER JOIN flights ON airports.airport_code = flights.arrival_airport
            INNER JOIN ticket_flights ON flights.flight_id = ticket_flights.flight_id
        WHERE
            airports.city = 'Москва'
            AND CAST(flights.departure_time AS date) BETWEEN '2018-07-23' AND '2018-09-30'
        GROUP BY
            week_number) t
    LEFT JOIN (
        SELECT
            festival_name,
            EXTRACT(week FROM festivals.festival_date) AS festival_week
        FROM
            festivals
        WHERE
            festival_city = 'Москва'
            AND CAST(festivals.festival_date AS date) BETWEEN '2018-07-23' AND '2018-09-30') t2 ON t.week_number = t2.festival_week) AS T;
'''